In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_def=pd.read_csv("data/Data_Dictionary.csv")
df_def




In [ ]:
df=pd.read_csv("data/Dataset.csv",low_memory=False)
df.head()

In [ ]:
# df.isna().count()
df.info()

In [ ]:
df.isna().sum()

In [ ]:
selected_columns=["Client_Income","Car_Owned","Bike_Owned","Active_Loan","House_Own","Child_Count","Credit_Amount","Loan_Annuity"]
df=df[selected_columns]
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df_new= df.copy(deep=True)

In [ ]:
df_new.dropna(subset=["Client_Income"],inplace=True)

In [ ]:
df_new[["Car_Owned","Bike_Owned","Active_Loan","House_Own","Child_Count"]].isna().sum()

In [ ]:
df_new[["Car_Owned","Bike_Owned","Active_Loan","House_Own","Child_Count"]]=df_new[["Car_Owned","Bike_Owned","Active_Loan","House_Own","Child_Count"]].fillna(0)

In [ ]:
df_new.isna().sum()

In [ ]:
df_new['Credit_Amount'].dtypes

In [ ]:
for column in df_new.columns:
    print(f"{column}: {df_new[column].dtype}")

In [ ]:
df_new['Credit_Amount'] = pd.to_numeric(df_new['Credit_Amount'], errors='coerce')
df_new['Client_Income'] = pd.to_numeric(df_new['Client_Income'], errors='coerce')

In [ ]:
df_new['Credit_Amount'].dtype

In [ ]:
df_new['Loan_Annuity'] = pd.to_numeric(df_new['Loan_Annuity'],errors='coerce')
df_new['Loan_Annuity'].dtype

In [ ]:
df_new['Loan_Annuity'].isna().sum()

In [ ]:
df_new['Credit_Amount'].isna().sum()

In [ ]:
mean_loan_annuity=df_new['Loan_Annuity'].mean()
mean_loan_annuity=round(mean_loan_annuity,2)
print(mean_loan_annuity)

In [ ]:
mean_credit_amount=df_new['Credit_Amount'].mean()
mean_credit_amount=round(mean_credit_amount,2)
print(mean_credit_amount)
df_new['Credit_Amount']=df_new['Credit_Amount'].fillna(mean_credit_amount)


In [ ]:
df_new.describe()

In [ ]:
data=df_new['Child_Count'].max()
data=df_new['Child_Count']
count=0
for i in df_new['Child_Count']:
    if i>6:
        count+=1
        print(i)
print(count)

In [ ]:
#Imputing the Child count as 6 if the value is greater than 6
df_new['Child_Count'] = df_new['Child_Count'].apply(lambda x: 6 if x > 6 else x)

In [ ]:
df_new.describe()

In [ ]:
df_new.isna().sum()

In [ ]:
from sklearn.linear_model import LinearRegression
X=df_new[['Credit_Amount']]
Y=df_new['Loan_Annuity'].fillna(0)

model = LinearRegression()

model.fit(X,Y)

In [ ]:
missing_loan_annuity_rows = df_new[df_new['Loan_Annuity'].isnull()]
missing_loan_annuity_rows

In [ ]:
predicted_loan_annuity = model.predict(missing_loan_annuity_rows[['Credit_Amount']])
df_new.loc[df_new['Loan_Annuity'].isnull(), 'Loan_Annuity'] = predicted_loan_annuity

In [ ]:
df_new.isna().sum()

In [ ]:

df_new.head()

In [ ]:


df_new.corr()


In [ ]:
syn_size = len(df_new)
df_new['Loan_Default'] = np.random.choice([0, 1], size=syn_size, p=[0.9, 0.1])


In [ ]:
df_new['Loan_Default'].value_counts()

In [ ]:
#Splitting features and traget var
x=df_new.drop(columns=['Loan_Default'])
y=df_new['Loan_Default']

In [ ]:
#Spliting train and test sets

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
# Using random forest

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_train,y_train)

In [177]:
#Evaluation of the model
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test,y_pred)
f1=f1_score(y_test,y_pred)
roc_auc = roc_auc_score(y_test,y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("ROC-AUC Score:", roc_auc)

Accuracy: 0.8785200845665962
Precision: 0.10363636363636364
Recall: 0.023389413212966764
F1-score: 0.0381653833277536
ROC-AUC Score: 0.5000744737304168


In [178]:
feature_importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': x_train.columns, 'Importance': feature_importances})
feature_importance_df

,Feature,Importance
0,Client_Income,0.216651
1,Car_Owned,0.006375
2,Bike_Owned,0.007824
3,Active_Loan,0.008590
4,House_Own,0.006288
5,Child_Count,0.021619
6,Credit_Amount,0.320890
7,Loan_Annuity,0.411763


In [ ]:
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.show()

In [176]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define hyperparameter distributions
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 10, 20],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, scoring='f1')

# Perform RandomizedSearchCV
random_search.fit(x_train, y_train)

# Get the best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best F1-score:", best_score)


Best Parameters: {'max_depth': None, 'min_samples_leaf': 3, 'min_samples_split': 9, 'n_estimators': 132}
Best F1-score: 0.00021090372245070126


In [179]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [180]:
with open('model.pkl', 'rb') as file:
    model=pickle.load(file)

In [197]:
client_income = 10000000
car_owned = 0
bike_owned = 0
active_loan = 0
house_own = 1
child_count = 2
credit_amount = 50000
loan_annuity = 2000


manual_input = pd.DataFrame({
    'Client_Income': [client_income],
    'Car_Owned': [car_owned],
    'Bike_Owned': [bike_owned],
    'Active_Loan': [active_loan],
    'House_Own': [house_own],
    'Child_Count': [child_count],
    'Credit_Amount': [credit_amount],
    'Loan_Annuity': [loan_annuity]
})


In [198]:
predictions = model.predict(manual_input)
print(predictions)

[0]


[0]
